В данном задании мне захотелось дообучить  модель на своей переписке из телеграма со своими друзьями,в итоге вышел датасет примерно да 10000 фраз совместо меня и друзей.
в качестве модкли для обучения я взял gpt2 и попытаюсь ее дообучить на своих фразах чтобы модель попробовала подражать моему стилю общения

In [58]:
import tensorflow as tf
import pandas as pd
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from keras.utils import to_categorical

from keras.utils import pad_sequences
from keras import models
from keras import Sequential
from keras import layers
from keras.callbacks import ModelCheckpoint
from keras import optimizers
from keras.metrics import AUC
from keras.regularizers import L1
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup 
import re
import torch
RANDOM_STATE = 42

print('Cuda version: ' + tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
file_path_test ="./dataset_token_labels.csv"
tokenized_datasets = pd.read_csv(file_path_test, encoding='utf-8')
tokenized_datasets.drop(columns=['tokenized_reviews', 'tokens'], inplace=True)

In [22]:
import pandas as pd
import json
import os

# Создайте пустой список для хранения данных из JSON файлов
dataframes = []

# Переберите каждый JSON файл в вашем каталоге
json_files = [file for file in os.listdir('.') if file.endswith('.json')]
for file in json_files:
    with open(file, 'r', encoding='utf-8') as f:
        # Загрузите данные из JSON файла
        data = json.load(f)
        # Преобразуйте данные в DataFrame и добавьте их в список
        df = pd.DataFrame(data)
        dataframes.append(df)

# Объедините все DataFrame в один
final_df = pd.concat(dataframes, ignore_index=True)

# Теперь у вас есть один Pandas DataFrame, содержащий данные из всех JSON файлов
print(final_df)


        name           type         id  \
0       V14D  personal_chat  200693762   
1       V14D  personal_chat  200693762   
2       V14D  personal_chat  200693762   
3       V14D  personal_chat  200693762   
4       V14D  personal_chat  200693762   
...      ...            ...        ...   
91578  V3N1K  personal_chat  360552278   
91579  V3N1K  personal_chat  360552278   
91580  V3N1K  personal_chat  360552278   
91581  V3N1K  personal_chat  360552278   
91582  V3N1K  personal_chat  360552278   

                                                messages  
0      {'id': 498643, 'type': 'service', 'date': '202...  
1      {'id': 498644, 'type': 'message', 'date': '202...  
2      {'id': 498645, 'type': 'message', 'date': '202...  
3      {'id': 498646, 'type': 'service', 'date': '202...  
4      {'id': 498647, 'type': 'message', 'date': '202...  
...                                                  ...  
91578  {'id': 778920, 'type': 'message', 'date': '202...  
91579  {'id': 778921, '

In [23]:
# Установите максимальное количество выводимых столбцов
pd.set_option('display.max_columns', None)

# Выведите данные final_df.head() в ширину максимально
final_df.head().style.set_properties(**{'width': '300px', 'text-align': 'left'})

,name,type,id,messages
0,V14D,personal_chat,200693762,"{'id': 498643, 'type': 'service', 'date': '2020-09-07T19:53:42', 'date_unixtime': '1599497622', 'actor': 'Tos', 'actor_id': 'user368720613', 'action': 'clear_history', 'text': '', 'text_entities': []}"
1,V14D,personal_chat,200693762,"{'id': 498644, 'type': 'message', 'date': '2020-09-07T19:55:36', 'date_unixtime': '1599497736', 'from': 'Tos', 'from_id': 'user368720613', 'text': 'Мощно', 'text_entities': [{'type': 'plain', 'text': 'Мощно'}]}"
2,V14D,personal_chat,200693762,"{'id': 498645, 'type': 'message', 'date': '2020-09-07T19:57:03', 'date_unixtime': '1599497823', 'from': 'Tos', 'from_id': 'user368720613', 'text': 'Влад?', 'text_entities': [{'type': 'plain', 'text': 'Влад?'}]}"
3,V14D,personal_chat,200693762,"{'id': 498646, 'type': 'service', 'date': '2020-09-07T20:00:44', 'date_unixtime': '1599498044', 'actor': 'Tos', 'actor_id': 'user368720613', 'action': 'phone_call', 'discard_reason': 'busy', 'text': '', 'text_entities': []}"
4,V14D,personal_chat,200693762,"{'id': 498647, 'type': 'message', 'date': '2020-09-07T20:10:09', 'date_unixtime': '1599498609', 'from': 'V14D', 'from_id': 'user200693762', 'text': 'Что ты хочешь Толя ?', 'text_entities': [{'type': 'plain', 'text': 'Что ты хочешь Толя ?'}]}"


In [25]:
import pandas as pd

# Предположим, что ваш DataFrame называется final_df

# Извлекаем значения "actor" и "text" из каждого словаря
final_df['actor'] = final_df['messages'].apply(lambda x: x.get('from', ''))
final_df['text'] = final_df['messages'].apply(lambda x: x.get('text', ''))

# Удаляем столбец "messages", так как он больше не нужен
final_df.drop(columns=['messages'], inplace=True)

# Выводим первые 5 строк нового датасета для проверки
print(final_df.head())



KeyError: 'messages'

In [26]:
print(final_df['messages'].iloc[0])

KeyError: 'messages'

In [29]:
final_df.head()

,name,type,id,actor,text
0,V14D,personal_chat,200693762,,
1,V14D,personal_chat,200693762,Tos,Мощно
2,V14D,personal_chat,200693762,Tos,Влад?
3,V14D,personal_chat,200693762,,
4,V14D,personal_chat,200693762,V14D,Что ты хочешь Толя ?


In [30]:
final_df.drop(columns=['name','type','id'], inplace=True)

In [31]:
final_df.to_csv('dataset.csv', index=False)

In [20]:
actor_values = final_df['actor'].unique()

# Выводим уникальные значения
print(actor_values)

['Tos' '' 'V14D' 'Xolod 2.0' 'IV4N0V']


In [1]:
print(final_df.head(10))

NameError: name 'final_df' is not defined

In [ ]:
final_df.to_csv('messager.csv', index=False)

In [11]:
file_path_test ="./messager.csv"
df = pd.read_csv(file_path_test, encoding='utf-8')


In [18]:
df_clean=df.dropna()

In [65]:
df_clean.head()

,actor,text
1,Tos,Мощно
2,Tos,Влад?
4,V14D,Что ты хочешь Толя ?
5,V14D,Я не понимаб
6,V14D,ну ок


In [67]:
text=df_clean["text"]
text.to_csv('messager2.csv', index=False)

In [9]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Загрузка модели casual language modeling
model_name = "GPT2"
model = GPT2LMHeadModel.from_pretrained(model_name)

# Загрузка токенизатора для модели
tokenizer = GPT2Tokenizer.from_pretrained(model_name)


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [23]:
texts = df_clean["text"].tolist()

In [25]:
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)

In [38]:
tokenizer = AutoTokenizer.from_pretrained("Dmitriy007/rugpt2_gen_news")
model = AutoModelForCausalLM.from_pretrained("Dmitriy007/rugpt2_gen_news")

pytorch_model.bin:   0%|          | 0.00/551M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

снизу код из hugging face по тому как использовать эту модель не дообучая ее ,она генерирует новости на русском языке

In [39]:
from transformers import AutoTokenizer, AutoModelForCausalLM



input_text = 'Ученик старшего класса лицея № 21 Иван Сидоров из города Адлер полетел в космос на планету Марс.'

inputs = tokenizer(input_text, return_tensors="pt")
model.to('cuda')
inputs.to('cuda')

input_ids = inputs["input_ids"]

output = model.generate(
input_ids,
attention_mask=inputs["attention_mask"],
pad_token_id=model.config.bos_token_id,
max_length=300,
num_beams=5,
num_return_sequences=1,
top_k=50,
top_p=0.90,
no_repeat_ngram_size=2,
temperature=0.7,
early_stopping=True
)

generated_text = list(map(tokenizer.decode, output))
print(generated_text[0])


c:\Users\anon\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\generation\configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\anon\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\generation\configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [68]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

# Загрузка предварительно обученного токенизатора и модели
tokenizer = AutoTokenizer.from_pretrained("Dmitriy007/rugpt2_gen_news")
model = AutoModelForCausalLM.from_pretrained("Dmitriy007/rugpt2_gen_news")

# Подготовка данных для дообучения
dataset = TextDataset(
    tokenizer=tokenizer,
    file_path="./messager2.csv",  # Путь к вашему файлу с данными
    block_size=128  # Максимальная длина последовательности
)

# Инициализация data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# Настройка аргументов обучения
training_args = TrainingArguments(
    output_dir="./output",  # Папка для сохранения модели
    overwrite_output_dir=True,
    num_train_epochs=3,  # Количество эпох обучения
    per_device_train_batch_size=8,
    save_steps=1000,
    save_total_limit=2
)

# Создание трейнера для обучения модели
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)

# Запуск обучения
trainer.train()

# Сохранение дообученной модели



c:\Users\anon\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\data\datasets\language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
c:\Users\anon\AppData\Local\Programs\Python\Python310\lib\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


  0%|          | 0/2541 [00:00<?, ?it/s]

TrainOutput(global_step=2541, training_loss=3.435218471750787, metrics={'train_runtime': 992.1285, 'train_samples_per_second': 20.489, 'train_steps_per_second': 2.561, 'train_loss': 3.435218471750787, 'epoch': 3.0})

In [90]:
model.save_pretrained("./model2")


In [84]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Загрузка токенизатора и модели
tokenizer = AutoTokenizer.from_pretrained("Dmitriy007/rugpt2_gen_news")
model = AutoModelForCausalLM.from_pretrained("./model2")

# это промпт
input_text = "Это странно"

# Токенизация 
inputs = tokenizer(input_text, return_tensors="pt")

# Передача модели на GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
inputs.to(device)

# Генерация текста
output = model.generate(
    inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    pad_token_id=tokenizer.eos_token_id,
    max_length=300,
    num_return_sequences=1,
    do_sample=True,  # Использовать режим с выборкой
    temperature=0.7,
    top_p=0.9,
    num_beams=5,  # Установить количество лучей больше 1
    early_stopping=False  # Отключить раннюю остановку
)


# Декодирование сгенерированного текста
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
 


In [88]:
generated_text

'Это странно\n"Да,это странно"\n"Да,это странно"\n"Да,это странно"\n"Да,это странно"\n"Да,это странно"\n"Да,это странно"\n"Да,это странно"\n"Да,это странно"\n"Да,это странно"\n"Да,это странно"\n"Да,это странно"\n"Да,это странно"\n"Да,это странно"\n"Да,это странно"\n"Да,это странно"\n"Да,это странно"\n"Да,это странно"\n"Да,это странно"\n"Да,это странно"\n"Да,это странно"\n"Да,это странно"\n"Да,это странно"\n"Да,это странно"\n"Да,это странно"\n"Да,это странно"\n"Да,это странно"\n"Да,это странно"\n"Да,это странно"\n"Да,это странно"\n"Да,это странно"\n"Да,это странно"\n"Да,это странно"\n"Да,это странно"\n"Да,это странно"\n"Да,это странно"\n"Да,это странно"\n"Да,это странно"\n"Да,это странно"\n"Да,это странно"\n"Да,это странно"\n"Да,это странно"\n"Да,это странно"\n"Да,'

Ну он определнно генерирует нечто похожее,на правду но мне совсем понятно почему в таком количестве

In [72]:
model
tokenizer


GPT2TokenizerFast(name_or_path='Dmitriy007/rugpt2_gen_news', vocab_size=50257, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	50257: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}